In [2]:
import numpy as np
import pandas as pd

In [3]:
data_file = '/home/student/Downloads/Accession Register-Books _with_ ISBN_numbers.csv'
data_table = pd.read_csv(data_file)
cols = ['Acc. Date', 'Acc. No.', 'Title', 'ISBN', 'Author/Editor', 'Ed./Vol.', 'Place & Publisher', 'Year', 'Page(s)', 'Class No./Book No.']
data_table = data_table[cols]
data_table.drop_duplicates(subset='ISBN', keep='first', inplace=True)
data_table

/tmp/ipykernel_20985/3140908523.py:2: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data_table = pd.read_csv(data_file)


,Acc. Date,Acc. No.,Title,ISBN,Author/Editor,Ed./Vol.,Place & Publisher,Year,Page(s),Class No./Book No.
0,20-01-2017,1,Network design : management and technical pers...,849334047,"Mann-Rubinson, Teresa C.",NaN,"Boca Raton: CRC Press,",1999,405 p.,NaN
1,08-09-2001,2,Multimedia information analysis and retrieval ...,9783540648260,"Ip, Horace H. S.",NaN,"Berlin: Springer,",1998,"viii, 264 p.;",004 IPH
2,08-09-2001,3,"Multimedia systems : delivering, generating, a...",1852332484,"Morris, Tim",NaN,"London: Springer,",2000,"xi, 191 p.;",006.7 MOR
3,05-09-2001,4,Principles of Data Mining and Knowledge Discovery,9783540410669,"Zytkov, Jan. M.",NaN,"New York: Springer-Verlag,",1999,593 p.,NaN
4,09-08-2001,5,Focusing solutions for data mining : analytica...,3540664297,"Reinartz, Thomas",NaN,"New York: Springer,",1999,"xiv, 307 p.;",006.3 REI
...,...,...,...,...,...,...,...,...,...,...
35916,05-01-2026,36354,The Ruba'iyat of Omar Khayyam,9780140443844,"Khayyam, Omar",NaN,"London : Penguin Books,",1981,116 p.,NaN
35917,05-01-2026,36355,Artificial intelligence for robotics,9781805129592,Francis X.,NaN,"UK : Packt Publishing,",2024,"xvii, 325 p. ;",006.3 FRA
35918,05-01-2026,36356,Too big to fail: The inside story of how wall ...,9780143118244,"Sorkin, Andrew Ross",NaN,"New York : Penguin Books,",2010,"xx, 618p. ;",330.973 SOR
35919,07-01-2026,36357,Digital communications : A foundational approach,9781009429665,"Fischer, Robert F. H.",NaN,"Cambridge : Cambridge University Press,",2024,"xiv, 397p. ;",621.382 FIS


In [4]:
from requests import request
from bs4 import BeautifulSoup
header = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:143.0) Gecko/20100101 Firefox/143.0'}

In [101]:
def summary_finder(isbn, header):
    isbn = str(isbn)
    if len(isbn) < 10:
        isbn = f"{'0'*(10 - len(isbn))}{isbn}"
    print(isbn)
    search_base_link = f'https://books.google.com/books?vid=ISBN{isbn}'
    search_req = request(method='GET', url=search_base_link, headers=header)
    book_soup = BeautifulSoup(search_req.text, 'html.parser')
    summary = book_soup.find(name='div', attrs={'id': 'synopsistext'})
    if not summary:
        summary = book_soup.find(name='div', attrs={'class': 'Mhmsgc'})
    for br in summary.find_all('br'):
        br.replace_with('\\n')
    if summary:
        return isbn, summary.text
    return isbn, ""
        

In [104]:
from concurrent.futures import ThreadPoolExecutor, as_completed

results = {}

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [
        executor.submit(summary_finder, isbn, header)
        for isbn in data_table['ISBN']
    ]

    for future in as_completed(futures):
        isbn, summary = future.result()
        results[isbn] = summary
        
        

0849334047
9783540648260
1852332484
9783540410669
3540664297
9783540660828
9783540660798
3540660445
3540661573
9783540664581
3540658661
9783540655169
354065304X
9783540659266
3540669841
1852330090
9783540668367
354057459X
3540658181
1852331402
3540655786
9783540619918
9783540657620
9783540667315
9780387987064
3540671943
9783540671961
9780387950372
3540640045
817808046X
8178080362
9812358676
9812359982
8178082314
8178080354
8178080389
8178081474
8178080370
8178083779
9814053864
817808094X
9814035890
9789814053228
8178080990
8178084287
8178080982
981235834X
8178081709
8178080907
8178081423
9780072119770
9780079137050
9780070499355
9780070412675
9780070246454
8120312538
8120309448
8120309251
8120312406
8120311183
8120315022
8120308654
8120316703
8120305329
8120307607
8120314778
8120308824
8120310187
8120307135
9788120310162
9780071188852
9780071004725
8178080877
9971513455
8173662657
8170297532
8187105550
9814066400
8178081458
8120310462
8170294991
1578700663
8178084252
8170298997
8120315

KeyboardInterrupt: 

0761994459
8170365422
8170361028
8170366925
9780761994206
9780803993020
8170364140
8170364906
8170365260
9780761993834
817036616X
8170369525
9780141006673
9788178810287
8178810379
0471497932
9780470841372
9780471988007
9780706918465
9780818683985
9780818683008
817033652X
8170336627
9780070472426
9780074633090
9780078824746
9780078824333
8178082349
9788178085401
9780130843173
0139601627
020175603X
817808001X
9780415916042
9780415926072
9780415095266
9780077103934
9780674006577
041525132X
9780415265942
0415040310
0415051347
9780566082764
9780745620114
9780393310344
9780262194648
9780415128094
9780415051828
041523963X
9780415236966
0521010772
9780521785808
9780192842213
9780192835130
019825055X
9780201767179
9780074630907
9780470202404
9780070944527
067400311X
9780262133982
1890951242
9780674043237
0745603866
9780745606095
9780435823818
817366286X
8173662509
8173660603
8173662541
8173660271
8173661162
8173662517
8173660662
8173660751
8173661278
8173660794
8173660336
8173661316
8173661332


In [ ]:
results

8176493228
1861972687
1861970544


{'1852332484': 'What are Multimedia Systems? This book is intended to expose you to multimedia in its widest sense. The popular understanding of this subject is that it is the integration and delivery of text, images and sound, usually on a desktop Pc. We are all familiar with examples from the extensive range of multimedia titles available on CD-ROM. These give us a rich set of sensory experiences, and, perhaps more important, we are free to navigate our way through the information as we choose. However, this is only half of the story since these titles can only deliver information. A recent IEEE publication (Chen, 1997) suggested that multimedia should include all systems that integrate diverse media, in standalone or networked applications, either for presentation or communication. Diverse media implies visual and aural information (text, static and moving pictures and sound). Networking could be realised using intranet or telephone links. Presentation systems would imply the multim

1861971184
1861971532
9780434915477
8170296242
9780672224850
9788170297680
9788183332033
5551234021622
9780917893162
9788176566261
8170294126
8170294134
8170294088
8176564702
8176564478
9788183331234
8170294193
9788183334587
5551234001577
9788183331593
9788183330985
9788183331357
9788176566353
9780830619092
8176564494
9780132346269
9780130280770
9780130871121
9780135721575
9780139491245
9780201809381
9780201325652
0201398567
9780201398595
020135084X
9780201618600
9780130619709
9780792376378
0792376064
9780792375685
079237598X
780792375753
9783540301462
9780792374282
9780792374138
0792374134
9780792373735
9781441948960
9780792386643
9780792386100
9780792385288
0792385071
9780792384694
9780792384601
9780792384175
9780792383604
9780792383611
9780792382812
9780792382072
9780792381860
9780792381389
9780792381167
9780792399940
9780792399384
9780792397441
9780140272499
9780387900926
9780201309935
0201309726
0201616173
020170921X
0201719622
0201360683
9780201821369
0201422190
9780534255183
053